# Introductory examples

## 1.usa.gov data from bit.ly

解析來自bit.ly的檔案，內容為，短網址服務的清單，有地點、瀏覽器資訊、時區等等不同的資訊，為json格式

In [ ]:
#顯示當前目錄位置
%pwd

#### 方法1.一次讀取一行解析

In [ ]:
path = 'ch02/usagov_bitly_data2012-03-16-1331923249.txt'

In [ ]:
open(path).readline()

#### 方法2.用隱含式轉換成list物件，一次拿到全部

In [ ]:
import json
path = 'ch02/usagov_bitly_data2012-03-16-1331923249.txt'

records = [json.loads(line) for line in open(path)]

In [ ]:
records[0]

In [ ]:
records[0]['tz']

In [ ]:
print(records[0]['tz'])

### Counting time zones in pure Python

#### 統計數據的方法們

In [ ]:
time_zones = [rec['tz'] for rec in records] #挖~並不是每項都有tz這屬性

In [ ]:
time_zones = [rec['tz'] for rec in records if 'tz' in rec]

In [ ]:
time_zones[:10]

#### 來統計一下個時區的數量!!!共有以下幾種作法。

#### 方法1. 採用迴圈配合字典

In [ ]:
def get_counts(sequence):
    counts = {}
    for x in sequence:
        if x in counts:
            counts[x] += 1
        else:
            counts[x] = 1
    return counts

print(get_counts(time_zones))

#### 方法2. 使用標準函式庫中的defaultdict方法，自動初始化字典物件value值

In [ ]:
from collections import defaultdict

def get_counts2(sequence):
    counts = defaultdict(int) # values will initialize to 0
    for x in sequence:
        counts[x] += 1
    return counts

print(get_counts(time_zones))

In [ ]:
counts = get_counts(time_zones)

In [ ]:
counts['America/New_York']

In [ ]:
len(time_zones)

In [ ]:
# 排序，並且預設前10項
def top_counts(count_dict, n=10):
    value_key_pairs = [(count, tz) for tz, count in count_dict.items()]
    value_key_pairs.sort()
    return value_key_pairs[:-n-1:-1]

In [ ]:
top_counts(counts)

#### 其實標準函式庫中有更方便的函數可以用

In [ ]:
from collections import Counter

In [ ]:
counts = Counter(time_zones)

In [ ]:
counts.most_common(10)

### Counting time zones with pandas

#### 更神奇的套件來啦!!!!

#### pandas為專門在做資料處理整理的套件，也是本書作者的作品

In [ ]:
#設定繪圖在jupyter上
%matplotlib inline

In [ ]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4)

In [ ]:
import json
path = 'ch02/usagov_bitly_data2012-03-16-1331923249.txt'
lines = open(path).readlines()
records = [json.loads(line) for line in lines]

#### 最有名的DataFrame功能來啦!!!
可以快速展示資料

In [ ]:
from pandas import DataFrame, Series
import pandas as pd

frame = DataFrame(records)
frame

In [ ]:
frame['tz'][:10]

In [ ]:
tz_counts = frame['tz'].value_counts()
tz_counts[:10]

In [ ]:
clean_tz = frame['tz'].fillna('Missing') #尋找NaN值
clean_tz[clean_tz == ''] = 'Unknown'
tz_counts = clean_tz.value_counts()
tz_counts[:10]

In [ ]:
frame['a'][1]

In [ ]:
frame['a'][50]

In [ ]:
frame['a'][51]

In [ ]:
plt.figure(figsize=(10, 4))

In [ ]:
tz_counts[9::-1].plot(kind='barh', rot=0)

#### 解析用戶習慣

萃取瀏覽器資訊

In [ ]:
results = Series([x.split()[0] for x in frame.a.dropna()]) #移除NaN值並且取出第一個單字
results[:5]

In [ ]:
results.value_counts()[:8]

#### 萃取出windows用戶與非windows用戶，並且使用時區區分

In [ ]:
cframe = frame[frame.a.notnull()]  #移除沒有a這項的資料

In [ ]:
operating_system = np.where(cframe['a'].str.contains('Windows'),
                            'Windows', 'Not Windows')
operating_system[:5]

In [ ]:
by_tz_os = cframe.groupby(['tz', operating_system])
by_tz_os.size().unstack()

In [ ]:
agg_counts = by_tz_os.size().unstack().fillna(0)
agg_counts[:10]

In [ ]:
# Use to sort in ascending order
indexer = agg_counts.sum(1).argsort() #對Y軸進行統計，並且回傳陣列的索引值

indexer[:10]

In [ ]:
count_subset = agg_counts.take(indexer)[-10:]
count_subset

In [ ]:
plt.figure()

In [ ]:
count_subset.plot(kind='barh', stacked=True)

In [ ]:
plt.figure()

In [ ]:
normed_subset = count_subset.div(count_subset.sum(1), axis=0)
normed_subset.plot(kind='barh', stacked=True)

## MovieLens 1M data set

分析100萬筆電影評分資料

* users.dat   [評分人id, 性別, 年紀, 職業, zip?]
* ratings.dat [評分人id, 電影id, 分數, 時間戳章]
* movies.dat  [電影id, 標題, 類型]

In [ ]:
import pandas as pd
import os
encoding = 'latin1'

#設定檔案路徑
upath = os.path.expanduser('ch02/movielens/users.dat')
rpath = os.path.expanduser('ch02/movielens/ratings.dat')
mpath = os.path.expanduser('ch02/movielens/movies.dat')

#設定欄位名稱
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
mnames = ['movie_id', 'title', 'genres']

#讀取，並設定分隔符號
users = pd.read_csv(upath, sep='::', header=None, names=unames, encoding=encoding, engine='python')
ratings = pd.read_csv(rpath, sep='::', header=None, names=rnames, encoding=encoding, engine='python')
movies = pd.read_csv(mpath, sep='::', header=None, names=mnames, encoding=encoding, engine='python')

In [ ]:
users[:5]

In [ ]:
ratings[:5]

In [ ]:
movies[:5]

In [ ]:
ratings

In [ ]:
#maerge起來，根據兩筆資料中相同名稱的欄位
data = pd.merge(pd.merge(ratings, users), movies)
data

In [ ]:
data.ix[0]

In [ ]:
#建立類似Eecel中的樞紐分析表功能，
#選擇計算欄位、和x,y軸，與計算方法
mean_ratings = data.pivot_table('rating', index='title',
                                columns='gender', aggfunc='mean')
mean_ratings[:5]

In [ ]:
#根據電影片名做數量統計
ratings_by_title = data.groupby('title').size()

In [ ]:
ratings_by_title[:5]

In [ ]:
#過濾區評分次數大於250次的電影
active_titles = ratings_by_title.index[ratings_by_title >= 250]

In [ ]:
active_titles[:10]

In [ ]:
#將最初統計的電影與性別統計結果過濾成評分次數250以上的電影清單

mean_ratings = mean_ratings.ix[active_titles]
mean_ratings

In [ ]:
#重新命名這部電影
mean_ratings = mean_ratings.rename(index={'Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)':
                           'Seven Samurai (Shichinin no samurai) (1954)'})

In [ ]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

### Measuring rating disagreement

#### 尋找男女喜好差異最大之電影

In [ ]:
#評分相減
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

In [ ]:
#排序
sorted_by_diff = mean_ratings.sort_values(by='diff')

#前15筆為女生喜愛大於男生差異最大的前15筆
sorted_by_diff[:15]

In [ ]:
# Reverse order of rows, take first 15 rows
#後15筆為男生喜愛大於女生差異最大的前15筆
sorted_by_diff[::-1][:15]

#### 找出評分差異最大的電影

In [ ]:
# Standard deviation of rating grouped by title
rating_std_by_title = data.groupby('title')['rating'].std() #計算各電影的標準差
# Filter down to active_titles
rating_std_by_title = rating_std_by_title.ix[active_titles] #取出大於250筆評分資料
# Order Series by value in descending order
rating_std_by_title.sort_values(ascending=False)[:10] #前10筆

### US Baby Names 1880-2010

#### 分析美國30年小孩命名資料

In [ ]:
from __future__ import division
from numpy.random import randn
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 5))
np.set_printoptions(precision=4)
%pwd

http://www.ssa.gov/oact/babynames/limits.html

In [ ]:
!head -n 10 ch02/names/yob1880.txt

In [ ]:
import pandas as pd
names1880 = pd.read_csv('ch02/names/yob1880.txt', names=['name', 'sex', 'births'])
names1880

In [ ]:
names1880.groupby('sex').births.sum() #統計性別

In [ ]:
#把30年全部讀進來吧!!!
# 2010 is the last available year right now
years = range(1880, 2011)

pieces = []
columns = ['name', 'sex', 'births']

for year in years:
    path = 'ch02/names/yob%d.txt' % year
    frame = pd.read_csv(path, names=columns)

    frame['year'] = year
    pieces.append(frame)

# Concatenate everything into a single DataFrame
names = pd.concat(pieces, ignore_index=True)

names

In [ ]:
#分析性別與年份關係
total_births = names.pivot_table('births', index='year',
                                 columns='sex', aggfunc=sum)

In [ ]:
total_births.tail() #.tail()為顯示最後幾筆資料

In [ ]:
#畫出來吧!!!
total_births.plot(title='Total births by sex and year')

In [ ]:
#統計名字出現率

def add_prop(group):
    # Integer division floors
    # births = group.births.astype(float) python3 不需要轉換
    # group['prop'] = births / births.sum()
    group['prop'] = group.births / group.births.sum()
    return group
names = names.groupby(['year', 'sex']).apply(add_prop)

In [ ]:
names

In [ ]:
#檢查加總是否接近1
np.allclose(names.groupby(['year', 'sex']).prop.sum(), 1)

In [ ]:
#取出每個group的前1000筆 方法1
def get_top1000(group):
    return group.sort_values(by='births', ascending=False)[:1000]
grouped = names.groupby(['year', 'sex'])
top1000 = grouped.apply(get_top1000)

In [ ]:
#取出每個group的前1000筆 方法2
pieces = []
for year, group in names.groupby(['year', 'sex']):
    pieces.append(group.sort_values(by='births', ascending=False)[:1000])
top1000 = pd.concat(pieces, ignore_index=True)

In [ ]:
#放置索引陣列
top1000.index = np.arange(len(top1000))

In [ ]:
top1000

### Analyzing naming trends

#### 分析命名趨勢

In [ ]:
#取出男女資料
boys = top1000[top1000.sex == 'M']
girls = top1000[top1000.sex == 'F']

In [ ]:
#建立年與名字的關係表
total_births = top1000.pivot_table('births', index='year', columns='name',
                                   aggfunc=sum)
total_births

In [ ]:
#畫出下列四個名字的趨勢圖
subset = total_births[['John', 'Harry', 'Mary', 'Marilyn']]
subset.plot(subplots=True, figsize=(12, 10), grid=False,
            title="Number of births per year")

#### Measuring the increase in naming diversity

#### 名字多樣性分析

In [ ]:
plt.figure()

In [ ]:
#繪製個年前1000個名稱流行度
table = top1000.pivot_table('prop', index='year',
                            columns='sex', aggfunc=sum)
table.plot(title='Sum of table1000.prop by year and sex',
           yticks=np.linspace(0, 1.2, 13), xticks=range(1880, 2020, 10))

#### 某年度需要多少名字數量才可以到達50%

In [ ]:
df = boys[boys.year == 2010]
df

In [ ]:
prop_cumsum = df.sort_values(by='prop', ascending=False).prop.cumsum() #建立序列
prop_cumsum[:10]

In [ ]:
prop_cumsum.values.searchsorted(0.5) + 1 #尋找第幾項才會加總道50%

#!!!!找出來後+1才是確切筆數，因為從0開始數

In [ ]:
#比較1990
df = boys[boys.year == 1900]
in1900 = df.sort_values(by='prop', ascending=False).prop.cumsum()
in1900.values.searchsorted(0.5) + 1

In [ ]:
#改寫成function
def get_quantile_count(group, q=0.5):
    group = group.sort_values(by='prop', ascending=False)
    return group.prop.cumsum().values.searchsorted(q) + 1

diversity = top1000.groupby(['year', 'sex']).apply(get_quantile_count)
diversity = diversity.unstack('sex')

diversity.head()

In [ ]:
diversity.plot(title="Number of popular names in top 50%")

#### The "Last letter" Revolution

#### 最後一個子母的變化

In [ ]:
#統計最後一個字母與性別年份的關係
# extract last letter from name column
get_last_letter = lambda x: x[-1]
last_letters = names.name.map(get_last_letter)
last_letters.name = 'last_letter'

table = names.pivot_table('births', index=last_letters,
                          columns=['sex', 'year'], aggfunc=sum)

In [ ]:
subtable = table.reindex(columns=[1910, 1960, 2010], level='year')
subtable.head()


In [ ]:
subtable.sum() #準備做正規化用

In [ ]:
letter_prop = subtable / subtable.sum().astype(float)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(10, 8))
letter_prop['M'].plot(kind='bar', rot=0, ax=axes[0], title='Male')
letter_prop['F'].plot(kind='bar', rot=0, ax=axes[1], title='Female', legend=False)

In [ ]:
plt.subplots_adjust(hspace=0.25)

In [ ]:
#選擇最多的三個字母進行全時間分析
letter_prop = table / table.sum().astype(float)

dny_ts = letter_prop.ix[['d', 'n', 'y'], 'M'].T
dny_ts.head()

In [ ]:
plt.close('all')

In [ ]:
dny_ts.plot()

#### Boy names that became girl names (and vice versa)

#### 男女變變變~有時候男女使用的名字，會有流行上的轉換。如Lesley 與 Leslie

In [ ]:
#尋找lesl開頭的單字
all_names = top1000.name.unique()
mask = np.array(['lesl' in x.lower() for x in all_names])
lesley_like = all_names[mask]
lesley_like

In [ ]:
#過濾出上述名字清單
filtered = top1000[top1000.name.isin(lesley_like)]
filtered.groupby('name').births.sum()

In [ ]:
#建立表，年份對性別，統計數量
table = filtered.pivot_table('births', index='year',
                             columns='sex', aggfunc='sum')
table = table.div(table.sum(1), axis=0)
table.tail()

In [ ]:
plt.close('all')

In [ ]:
table.plot(style={'M': 'k-', 'F': 'k--'})